In [1]:
import os
import argparse

import torch
import os
import argparse
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from os2d.modeling.model import build_os2d_from_config
from os2d.config import cfg
import  os2d.utils.visualization as visualizer
from os2d.structures.feature_map import FeatureMapSize
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio
from os2d.data import dataloader
from os2d.modeling.model import build_os2d_from_config

from os2d.data.dataloader import build_eval_dataloaders_from_cfg, build_train_dataloader_from_config
from os2d.engine.train import trainval_loop
from os2d.utils import set_random_seed, get_trainable_parameters, mkdir, save_config, setup_logger, get_data_path
from os2d.engine.optimization import create_optimizer
from os2d.config import cfg
from os2d.utils.visualization import *
import random
import os2d.utils.visualization as visualizer
from pathlib import Path
import cv2
import numpy as np
from os2d.utils import get_image_size_after_resize_preserving_aspect_ratio
from src.util.detection import generate_detection_boxes
from src.util.visualize import visualize_boxes_on_image
from src.util.filter import DataLoaderDB

In [2]:
if cfg.is_cuda:
    assert torch.cuda.is_available(), "Do not have available GPU, but cfg.is_cuda == 1"
    torch.backends.cudnn.benchmark = True

# random seed
set_random_seed(cfg.random_seed, cfg.is_cuda)

# Model
net, box_coder, criterion, img_normalization, optimizer_state = build_os2d_from_config(cfg)

# Optimizer
parameters = get_trainable_parameters(net)
optimizer = create_optimizer(parameters, cfg.train.optim, optimizer_state)

# load the dataset
data_path = get_data_path()
dataloader_train, datasets_train_for_eval = build_train_dataloader_from_config(cfg, box_coder, img_normalization,
                                                                                data_path=data_path)

dataloaders_eval = build_eval_dataloaders_from_cfg(cfg, box_coder, img_normalization,
                                                    datasets_for_eval=datasets_train_for_eval,
                                                    data_path=data_path)

db = DataLoaderDB( path = './src/db/data.csv' , dataloader = dataloader_train)

### Test Basic Method of DB

In [3]:
image_ids = list ( map( int , db.get_image_ids()) )
sorted_image_ids = sorted(image_ids)
print( sorted_image_ids )

[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 201, 202, 203, 207, 209, 210, 212, 213, 214, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 2

In [4]:
db.get_class_ids_by_image_id(0)

{'0': 3, '1': 3, '2': 2, '3': 4, '4': 4, '5': 3, '6': 3, '7': 2, '1055': 6}

In [5]:
len(db.get_value_by_id( 10 , 42 ))

13

### Test Basic Method of ContextAoiAlign

In [6]:
from src.lcp.ct_aoi_align import ContextAoiAlign
transform_image = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(img_normalization["mean"], img_normalization["std"])
                      ])

context_aoi_align = ContextAoiAlign( db, dataloader_train, transform_image , net , cfg )

In [7]:
# context_aoi_align.compute_roi_region_for_all()

In [8]:
context_aoi_align.get_feature_map( 0 )

tensor([[[2.6359, 0.7501, 0.5805,  ..., 0.2295, 0.4535, 1.1069],
         [0.6802, 0.3717, 0.0000,  ..., 0.2353, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2768, 0.5063, 0.0000,  ..., 0.3253, 0.7125, 0.0000],
         [0.3378, 0.0000, 0.0000,  ..., 0.0000, 0.7866, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0829, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2380, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 2.5919, 3.4586, 0.4234],
         [0.0000, 0.0000, 0.0000,  ..., 0.7022, 1.2554, 0.0000]],

        [[0.0000, 0.6587, 0.0000,  ..., 0.0000, 1.6693, 2.2038],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.3089, 1.7788],
         [0.0000, 0.3178, 0.0000,  ..., 0.0000, 0.9639, 1.

In [9]:
context_aoi_align.get_feature_map( 0 ).shape

torch.Size([1024, 82, 109])

In [10]:
context_aoi_align.get_feature_map_no_grad( 0 )

tensor([[[2.6359, 0.7501, 0.5805,  ..., 0.2295, 0.4535, 1.1069],
         [0.6802, 0.3717, 0.0000,  ..., 0.2353, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2768, 0.5063, 0.0000,  ..., 0.3253, 0.7125, 0.0000],
         [0.3378, 0.0000, 0.0000,  ..., 0.0000, 0.7866, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0829, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2380, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 2.5919, 3.4586, 0.4234],
         [0.0000, 0.0000, 0.0000,  ..., 0.7022, 1.2554, 0.0000]],

        [[0.0000, 0.6587, 0.0000,  ..., 0.0000, 1.6693, 2.2038],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.3089, 1.7788],
         [0.0000, 0.3178, 0.0000,  ..., 0.0000, 0.9639, 1.

In [11]:
context_aoi_align.get_feature_map_no_grad( 0 ).shape

torch.Size([1024, 82, 109])

In [12]:
# context_aoi_align.extract_roi_features_by_ids( 0 , 0 )

In [13]:
context_aoi_align.extract_roi_features_by_ids_in_no_grad( 0, 0 )

[{'image_id': 0,
  'class_id': 0,
  'truth_box': ((350.8800048828125, 0.0),
   (767.0399780273438, 595.598388671875)),
  'context_roi': ((350.8800048828125, 0.0),
   (783.6328192660219, 626.5802359988452)),
  'features': {'truth_feature': tensor([[[4.2222e-01, 0.0000e+00, 5.6205e-02,  ..., 7.1005e-01,
             6.0172e-01, 1.6310e+00],
            [4.9514e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
             4.1607e-02, 5.1353e-01],
            [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
             0.0000e+00, 3.1874e-02],
            ...,
            [1.5927e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
             0.0000e+00, 0.0000e+00],
            [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
             0.0000e+00, 0.0000e+00],
            [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
             0.0000e+00, 0.0000e+00]],
   
           [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
             0.0000e+00, 0.0000e+00],
       

In [14]:
context_aoi_align.roi_align( 0 , 0 , 
                            (350.8800048828125, 0.0) , (767.0399780273438, 595.598388671875), 
                            (350.8800048828125, 0.0) , (783.6328192660219, 626.5802359988452) 
                            )

{'truth_feature': tensor([[[4.2222e-01, 0.0000e+00, 5.6205e-02,  ..., 7.1005e-01,
           6.0172e-01, 1.6310e+00],
          [4.9514e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           4.1607e-02, 5.1353e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.1874e-02],
          ...,
          [1.5927e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000

In [15]:
context_aoi_align.extract_roi_align_no_grad( 0 , 0 , 
                                            (350.8800048828125, 0.0) , (767.0399780273438, 595.598388671875), 
                                            (350.8800048828125, 0.0) , (783.6328192660219, 626.5802359988452) 
                                            )

{'truth_feature': tensor([[[4.2222e-01, 0.0000e+00, 5.6205e-02,  ..., 7.1005e-01,
           6.0172e-01, 1.6310e+00],
          [4.9514e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           4.1607e-02, 5.1353e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.1874e-02],
          ...,
          [1.5927e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000

In [16]:
# context_aoi_align.extract_roi_features_for_all()

In [17]:
from src.lcp.aux_net import AuxiliaryNetwork
aux_net = AuxiliaryNetwork( context_aoi_align, db )

In [18]:
point1_x = float( db.get_specific_data(0 , 0 , 'point1_x' )[0] )
point1_y = float( db.get_specific_data(0 , 0 , 'point1_y' )[0] )
point2_x = float( db.get_specific_data(0 , 0 , 'point2_x' )[0] )
point2_y = float( db.get_specific_data(0 , 0 , 'point2_y' )[0] )   

point1 = ( point1_x , point1_y )
point2 = ( point2_x , point2_y )
image_id = 0
class_id = 0 
m = 5

In [19]:
print("=== 測試回歸損失近似函數 ===")
ar_loss = aux_net.approximate_ar_loss_no_grad(image_id, class_id, point1, point2, m = 3)
print(f"Image {image_id}, Class {class_id}: AR Loss = {ar_loss:.4f}")
print(f"返回類型: {type(ar_loss)}")
print(f"數值檢查: {'✓ 正常' if ar_loss >= 0 else '✗ 異常'}")

=== 測試回歸損失近似函數 ===
Image 0, Class 0: AR Loss = 0.8786
返回類型: <class 'float'>
數值檢查: ✓ 正常


In [20]:
print("\n=== 測試分類損失近似函數 ===")
ac_loss = aux_net.approximate_ac_loss_no_grad(image_id, class_id, point1, point2)
print(f"Image {image_id}, Class {class_id}: AC Loss = {ac_loss:.4f}")
print(f"返回類型: {type(ac_loss)}")
print(f"數值檢查: {'✓ 正常' if ac_loss >= 0 else '✗ 異常'}")


=== 測試分類損失近似函數 ===
Image 0, Class 0: AC Loss = 0.4542
返回類型: <class 'float'>
數值檢查: ✓ 正常


In [21]:
# 測試 Context RoI Align 整合
print("\n=== 測試 Context RoI Align 整合 ===")

# 1. 測試 Context ROI 獲取
context_roi_point1, context_roi_point2 = aux_net._context_roi_align._get_roi_region_specific(
    image_id, class_id, point1, point2
)
print(f"Context ROI: {context_roi_point1} -> {context_roi_point2}")

# 2. 測試特徵提取
contextual_features = aux_net._context_roi_align.extract_roi_align_no_grad(
    image_id, class_id, point1, point2,
    context_roi_point1, context_roi_point2
)

if contextual_features:
    print(f"特徵提取: ✓ 成功")
    combined_feature = contextual_features['combined_feature']
    print(f"特徵維度: {combined_feature.shape}")
    
    import torch
    print(f"特徵統計: mean={torch.mean(combined_feature):.4f}, std={torch.std(combined_feature):.4f}")
else:
    print("特徵提取: ✗ 失敗")



=== 測試 Context RoI Align 整合 ===
Context ROI: (350.8800048828125, 0.0) -> (783.6328192660219, 626.5802359988452)
特徵提取: ✓ 成功
特徵維度: torch.Size([1024, 7, 7])
特徵統計: mean=0.8192, std=1.5051


In [22]:
# 性能測試
import time

print("\n=== 性能測試 ===")

# 測試執行時間
iterations = 5
start_time = time.time()

for i in range(iterations):
    ar_loss = aux_net.approximate_ar_loss_no_grad(image_id, class_id, point1, point2)
    ac_loss = aux_net.approximate_ac_loss_no_grad(image_id, class_id, point1, point2)

end_time = time.time()
avg_time = (end_time - start_time) / iterations

print(f"平均執行時間: {avg_time:.4f} 秒")
print(f"執行 {iterations} 次總時間: {end_time - start_time:.4f} 秒")



=== 性能測試 ===
平均執行時間: 10.3103 秒
執行 5 次總時間: 51.5513 秒


In [23]:
# 詳細調試
print("\n=== 詳細調試執行 ===")

print(f"輸入參數: image_id={image_id}, class_id={class_id}")
print(f"邊界框: {point1} -> {point2}")

# 步驟 1: Context ROI 獲取
print("\n步驟 1: Context ROI 獲取")
context_roi_point1, context_roi_point2 = aux_net._context_roi_align._get_roi_region_specific(
    image_id, class_id, point1, point2
)
print(f"結果: {context_roi_point1} -> {context_roi_point2}")

# 步驟 2: 特徵提取
print("\n步驟 2: 特徵提取")
contextual_features = aux_net._context_roi_align.extract_roi_align_no_grad(
    image_id, class_id, point1, point2,
    context_roi_point1, context_roi_point2
)

if contextual_features:
    combined_feature = contextual_features['combined_feature']
    print(f"特徵維度: {combined_feature.shape}")
    
    import torch
    global_mean = torch.mean(combined_feature)
    global_std = torch.std(combined_feature)
    print(f"全域統計: mean={global_mean:.4f}, std={global_std:.4f}")
    
    # 步驟 3: 統計計算
    print("\n步驟 3: 統計計算")
    C, H, W = combined_feature.shape
    channel_means = torch.mean(combined_feature.view(C, -1), dim=1)
    mean_deviations = torch.abs(channel_means - global_mean)
    localization_sensitivity = torch.mean(mean_deviations) / (global_std + 1e-8)
    print(f"定位敏感性: {localization_sensitivity:.4f}")

# 步驟 4: GIoU 統計
print("\n步驟 4: GIoU 統計")
giou_stats = aux_net._compute_giou_statistics(image_id, class_id)
print(f"GIoU 統計: {giou_stats}")

# 步驟 5: 最終損失計算
print("\n步驟 5: 最終損失計算")
ar_loss = aux_net.approximate_ar_loss_no_grad(image_id, class_id, point1, point2, m = 3)
ac_loss = aux_net.approximate_ac_loss_no_grad(image_id, class_id, point1, point2)
print(f"AR Loss: {ar_loss:.4f}")
print(f"AC Loss: {ac_loss:.4f}")
print(f"Total: {ar_loss + ac_loss:.4f}")



=== 詳細調試執行 ===
輸入參數: image_id=0, class_id=0
邊界框: (350.8800048828125, 0.0) -> (767.0399780273438, 595.598388671875)

步驟 1: Context ROI 獲取
結果: (350.8800048828125, 0.0) -> (783.6328192660219, 626.5802359988452)

步驟 2: 特徵提取
特徵維度: torch.Size([1024, 7, 7])
全域統計: mean=0.8192, std=1.5051

步驟 3: 統計計算
定位敏感性: 0.4103

步驟 4: GIoU 統計
GIoU 統計: {'mean': 0.8587324282294295, 'std': 0.04220323204537694, 'count': 3}

步驟 5: 最終損失計算
AR Loss: 0.8786
AC Loss: 0.4542
Total: 1.3329


#### Test for LCP

In [24]:
from src.lcp.lcp import LCP


In [25]:
lcp = LCP(net, aux_net, dataloader_train)
lcp.init_for_indices()

[LCP] 初始化完成，共 43 層的 channel 索引


In [26]:
img_tensor = lcp.get_image_tensor_from_dataloader(image_id=0)


In [27]:
layers = lcp.get_layers_name()
for name, ch in layers:
    print(f"{name}: {ch} channels")

conv1: 64 channels
layer1.0.conv1: 64 channels
layer1.0.conv2: 64 channels
layer1.0.conv3: 256 channels
layer1.0.downsample.0: 256 channels
layer1.1.conv1: 64 channels
layer1.1.conv2: 64 channels
layer1.1.conv3: 256 channels
layer1.2.conv1: 64 channels
layer1.2.conv2: 64 channels
layer1.2.conv3: 256 channels
layer2.0.conv1: 128 channels
layer2.0.conv2: 128 channels
layer2.0.conv3: 512 channels
layer2.0.downsample.0: 512 channels
layer2.1.conv1: 128 channels
layer2.1.conv2: 128 channels
layer2.1.conv3: 512 channels
layer2.2.conv1: 128 channels
layer2.2.conv2: 128 channels
layer2.2.conv3: 512 channels
layer2.3.conv1: 128 channels
layer2.3.conv2: 128 channels
layer2.3.conv3: 512 channels
layer3.0.conv1: 256 channels
layer3.0.conv2: 256 channels
layer3.0.conv3: 1024 channels
layer3.0.downsample.0: 1024 channels
layer3.1.conv1: 256 channels
layer3.1.conv2: 256 channels
layer3.1.conv3: 1024 channels
layer3.2.conv1: 256 channels
layer3.2.conv2: 256 channels
layer3.2.conv3: 1024 channels
layer

In [28]:
layer_name = 'net_feature_maps.layer3.2.conv2'  # 根據您的網路架構調整
lambda_rate = 1.0
use_image_num = 3  # 使用少量圖像進行測試

print(f"測試層: {layer_name}")
print(f"Lambda 率: {lambda_rate}")
print(f"使用圖像數: {use_image_num}")

測試層: net_feature_maps.layer3.2.conv2
Lambda 率: 1.0
使用圖像數: 3


In [29]:
# 檢查設備一致性
print("=== 設備一致性檢查 ===")

# 檢查網路設備
net_device = next(lcp._net.parameters()).device
prune_net_device = next(lcp._prune_net.parameters()).device

print(f"主網路設備: {net_device}")
print(f"剪枝網路設備: {prune_net_device}")

# 檢查圖像張量設備
img_tensor = lcp.get_image_tensor_from_dataloader(0, is_cuda=False)
print(f"圖像張量設備: {img_tensor.device}")


=== 設備一致性檢查 ===
主網路設備: cpu
剪枝網路設備: cpu
圖像張量設備: cuda:0


In [30]:
# # 執行無梯度通道重要性計算
# print("\n=== 執行無梯度通道重要性計算 ===")

# try:
#     importance_scores = lcp.compute_channel_importance_no_grad(
#         layer_name=layer_name,
#         lambda_rate=lambda_rate,
#         use_image_num=use_image_num,
#         random_seed=42
#     )
    
#     print(f"✅ 計算成功！")
#     print(f"通道數量: {len(importance_scores)}")
#     print(f"重要性分數範圍: {np.min(importance_scores):.4f} - {np.max(importance_scores):.4f}")
#     print(f"平均重要性: {np.mean(importance_scores):.4f}")
#     print(f"標準差: {np.std(importance_scores):.4f}")
    
#     # 顯示前10個通道的重要性
#     print(f"\n前10個通道重要性分數:")
#     for i in range(min(10, len(importance_scores))):
#         print(f"  通道 {i}: {importance_scores[i]:.4f}")
        
# except Exception as e:
#     print(f"❌ 計算失敗: {e}")
#     import traceback
#     traceback.print_exc()


In [31]:
# # 測試各個組件功能
# print("\n=== 詳細組件測試 ===")

# # 測試圖像數據準備
# image_id_all = list(map(int, lcp._get_db().get_image_ids()))
# print(f"總圖像數: {len(image_id_all)}")
# print(f"前5個圖像ID: {image_id_all[:5]}")

# # 測試單個圖像的特徵統計計算
# test_image_id = image_id_all[0]
# print(f"\n測試圖像 {test_image_id} 的特徵統計:")

# try:
#     feature_stats = lcp._extract_and_compute_theoretical_statistics(test_image_id, layer_name)
    
#     if feature_stats:
#         print(f"✅ 特徵統計計算成功")
#         print(f"通道數: {len(feature_stats['channels'])}")
#         print(f"全域統計: mean={feature_stats['global_mean']:.4f}, std={feature_stats['global_std']:.4f}")
        
#         # 顯示前3個通道的詳細統計
#         print(f"\n前3個通道詳細統計:")
#         for i in range(min(3, len(feature_stats['channels']))):
#             ch_stat = feature_stats['channels'][i]
#             print(f"  通道 {i}:")
#             print(f"    L1範數: {ch_stat['l1_norm']:.4f}")
#             print(f"    方差: {ch_stat['variance']:.4f}")
#             print(f"    均值偏差: {ch_stat['mean_deviation']:.4f}")
#             print(f"    能量: {ch_stat['energy']:.4f}")
#             print(f"    稀疏性: {ch_stat['sparsity']:.4f}")
#             print(f"    重要性: {ch_stat['importance']:.4f}")
#     else:
#         print("❌ 特徵統計計算失敗")
        
# except Exception as e:
#     print(f"❌ 特徵統計測試失敗: {e}")


In [32]:
# # 驗證數學公式實現
# print("\n=== 數學公式驗證 ===")

# if feature_stats:
#     # 手動驗證統合公式
#     test_channel = feature_stats['channels'][0]
#     global_std = feature_stats['global_std']
    
#     # 手動計算重要性分數
#     manual_importance = (
#         0.3 * (test_channel['l1_norm'] / global_std) +                    # w1×L1/σ
#         0.2 * (test_channel['variance'] / (global_std ** 2)) +            # w2×Var/σ²
#         0.2 * (test_channel['mean_deviation'] / global_std) +             # w3×|μ-μ_g|/σ
#         0.15 * (test_channel['energy'] / (global_std ** 2)) +             # w4×E/σ²
#         0.15 * (1.0 - test_channel['sparsity'])                           # w5×(1-sparsity)
#     )
    
#     computed_importance = test_channel['importance']
    
#     print(f"手動計算重要性: {manual_importance:.6f}")
#     print(f"函數計算重要性: {computed_importance:.6f}")
#     print(f"差異: {abs(manual_importance - computed_importance):.8f}")
#     print(f"公式驗證: {'✅ 正確' if abs(manual_importance - computed_importance) < 1e-6 else '❌ 錯誤'}")


In [33]:
# # 性能測試
# print("\n=== 性能測試 ===")

# import time
# import psutil
# import os

# process = psutil.Process(os.getpid())

# # 測試執行時間和記憶體使用
# start_memory = process.memory_info().rss / 1024 / 1024  # MB
# start_time = time.time()

# # 執行多次測試
# test_iterations = 3
# for i in range(test_iterations):
#     print(f"執行第 {i+1} 次測試...")
#     importance_scores = lcp.compute_channel_importance_no_grad(
#         layer_name=layer_name,
#         lambda_rate=lambda_rate,
#         use_image_num=2,  # 減少圖像數以加快測試
#         random_seed=42
#     )

# end_time = time.time()
# end_memory = process.memory_info().rss / 1024 / 1024  # MB

# avg_time = (end_time - start_time) / test_iterations
# memory_usage = end_memory - start_memory

# print(f"\n性能統計:")
# print(f"  平均執行時間: {avg_time:.4f} 秒")
# print(f"  記憶體使用增量: {memory_usage:.2f} MB")
# print(f"  總執行時間: {end_time - start_time:.4f} 秒")


In [34]:
# # 通道重要性排序測試
# print("\n=== 通道排序測試 ===")

# if 'importance_scores' in locals():
#     # 獲取重要性排序
#     sorted_indices = np.argsort(importance_scores)[::-1]  # 降序排列
    
#     print(f"最重要的前10個通道:")
#     for i in range(min(10, len(sorted_indices))):
#         channel_idx = sorted_indices[i]
#         importance = importance_scores[channel_idx]
#         print(f"  排名 {i+1}: 通道 {channel_idx} (重要性: {importance:.4f})")
    
#     print(f"\n最不重要的後5個通道:")
#     for i in range(max(0, len(sorted_indices)-5), len(sorted_indices)):
#         channel_idx = sorted_indices[i]
#         importance = importance_scores[channel_idx]
#         rank = len(sorted_indices) - i
#         print(f"  倒數第 {rank}: 通道 {channel_idx} (重要性: {importance:.4f})")
    
#     # 統計分析
#     top_10_percent = int(len(importance_scores) * 0.1)
#     top_scores = importance_scores[sorted_indices[:top_10_percent]]
#     bottom_scores = importance_scores[sorted_indices[-top_10_percent:]]
    
#     print(f"\n統計分析:")
#     print(f"  前10%通道平均重要性: {np.mean(top_scores):.4f}")
#     print(f"  後10%通道平均重要性: {np.mean(bottom_scores):.4f}")
#     print(f"  重要性比率: {np.mean(top_scores) / np.mean(bottom_scores):.2f}")


In [35]:
# # 測試總結
# print("\n" + "="*60)
# print("🎯 LCP 無梯度通道重要性計算測試總結")
# print("="*60)

# test_results = {
#     '基礎計算': 'importance_scores' in locals() and len(importance_scores) > 0,
#     '特徵統計': 'feature_stats' in locals() and feature_stats is not None,
#     '數學公式': 'manual_importance' in locals(),
#     '性能測試': 'avg_time' in locals(),
#     '排序分析': 'sorted_indices' in locals()
# }

# print("測試項目結果:")
# for test_name, result in test_results.items():
#     status = "✅ 通過" if result else "❌ 失敗"
#     print(f"  {test_name}: {status}")

# overall_success = all(test_results.values())
# print(f"\n🏆 整體測試結果: {'🎉 全部通過！' if overall_success else '⚠️ 部分項目需要檢查'}")

# if overall_success:
#     print("\n✨ 您的 LCP 無梯度通道重要性計算實現完全正確！")
#     print("📊 統合數學公式正確實現")
#     print("🚀 無梯度架構完美整合")
#     print("🎯 準備好進行實際的通道剪枝應用！")
# else:
#     print("\n🔧 請檢查失敗的測試項目並進行相應調整。")


In [36]:
# # 例：剪掉 layer3.2.conv2 一半通道
# keep, discard = lcp.get_channel_selection_by_no_grad(
#     layer_name   = "net_feature_maps.layer3.2.conv2",
#     discard_rate = 0.5,
#     lambda_rate  = 1.0,
#     use_image_num= 5,
#     random_seed  = 42
# )

# print("保留通道:", keep)
# print("捨棄通道:", discard)


### Create DB for keep indices

In [37]:
from src.util.prune_db import PruneDBControler
prune_db = PruneDBControler( path = './src/db/prune_channel_information.csv' )
prune_db.initial()

檔案 ./src/db/prune_channel_information.csv 已存在，將刪除並重新創建。
創建新檔案: ./src/db/prune_channel_information.csv
檔案 ./src/db/prune_channel_information.csv 初始化完成。


In [38]:

layers = lcp.get_layers_name()
for name, ch in layers:
    if name != 'layer2.1.conv2':
        continue
    print(f"{name}: {ch} channels")
    keep, discard = lcp.get_channel_selection_by_no_grad(
        layer_name   = f"net_feature_maps.{name}",
        discard_rate = 0.5,
        lambda_rate  = 1.0,
        use_image_num= 3,
        random_seed  = 42
    )
    print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")
    prune_db.write_data(
        layer = f"net_feature_maps.{name}",
        original_channel_num= ch,
        num_of_keep_channel = len(keep),
        keep_index  = keep
    )
    break


layer2.1.conv2: 128 channels
[LCP] 開始基於數學推導的無梯度通道重要性計算 - net_feature_maps.layer2.1.conv2


KeyboardInterrupt: 

### Test for Pruner

In [ ]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )
dependencies = pruner.resolve_layer_dependencies('net_feature_maps.layer2.1.conv2')
print("依賴關係分析結果:")
print(f"目標層級: {dependencies['target']}")
print(f"BatchNorm 層: {dependencies['batch_norm']}")
print(f"下游層級: {dependencies['downstream_layers']}")
print(f"跳躍連接: {dependencies['skip_connections']}")
print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )
dependencies = pruner.resolve_layer_dependencies('net_feature_maps.layer2.1.conv3')
print("依賴關係分析結果:")
print(f"目標層級: {dependencies['target']}")
print(f"BatchNorm 層: {dependencies['batch_norm']}")
print(f"下游層級: {dependencies['downstream_layers']}")
print(f"跳躍連接: {dependencies['skip_connections']}")
print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )
for name, ch in layers:
    dependencies = pruner.resolve_layer_dependencies(f'net_feature_maps.{name}')
    print("依賴關係分析結果:")
    print(f"目標層級: {dependencies['target']}")
    print(f"BatchNorm 層: {dependencies['batch_norm']}")
    print(f"下游層級: {dependencies['downstream_layers']}")
    print(f"跳躍連接: {dependencies['skip_connections']}")
    print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:

layer_name = 'net_feature_maps.layer2.1.conv2'  # 根據您的網路架構調整
keep_indexes = prune_db.get_layer_keep_indices(layer_name)

# 執行輸出通道剪枝
success = pruner._prune_out_channel(layer_name, keep_indexes)

if success:
    print("✅ 輸出通道剪枝成功")
else:
    print("❌ 輸出通道剪枝失敗")
